1) gangnamOutput-modify.csv에서 지역만 얻어내기

In [2]:
import pandas as pd
data = pd.read_csv('/Users/junho/Hadoop-Classification/data/crawling/gangnamOutput-modify.csv')
data.describe()
unique_regions = data['region'].unique().tolist()
restaurant_list = data['name'].tolist()
unique_regions = data['region'].str.replace('서울 ', '', regex=False).unique().tolist()
print(unique_regions)
print(restaurant_list)

['마포', '영등포', '성동', '강북', '관악', '송파', '강서', '용산', '강남', '광진', '서초', '서대문', '동작', '양천', '금천', '은평', '중구', '성북', '종로', '동대문', '중랑', '강동', '노원', '도봉']
['살쾡이', '모에루 문래본점', 'HDD피자', '와와 쪽갈비 수유점', '소나무뜰끌림부추삼겹살', '백정집', '이자카야 토라 수유점', '마미떡 송파점', '제이클레프', '장효서울', '중화식', '수다 청담점', '153 구포국수 군자점', '하우마라탕', '우리돼지 솥뚜껑 삼겹살', '등촌샤브칼국수 사당역점', '영동소곱창', '루프탑미드', '하남돼지집 서울대입구역점', '신용산진배기고기집', '트리플에이커피 봉천e편한세상점', 'HDD판다', '쿠라우니', '청담오후', '포도젝트', '시온텐동', '수다 서래점', '신촌양꼬치', '막홀릭하우스', '임실순대국', '우뚝1984', '카페아뜨레', '더 가즈믹 테이블', '카페 더하리', '하마루', '주술소', '꽃보다크레페', '오목교곱창 목동본점', '라모따', '팔러 서울', '이자카야 명호', '제일곱창 연신내직영점', '트리블리 와인바', '유타로', '이천면옥함흥냉면', '갈매기의꿈', '하라고지페', '스시신조', '매료쪽갈비찜 송파본점', '곰팔이생고기,돼지갈비', '온량(장위동)', '호랭이술상', '김봉남포장마차 가로수길근처점', '미스터쭈꾸미', '빅파이브 종로점', '이태원교집합', '대물장어', '사조참치 장한평점', '향만루', '더양평해장국 성수역점', '조옥당 약수본점', '오목교별장', '희객 2호점', '켄홈 방이점', 'After Work33', '연남토마 성수점', '주한상', '우미마토', '달길포차', '뜸들이다 시흥점', '스트라이프', '골짝마을 닭한마리', '오늘튀김', '이여곰탕 마포점', '언필드', '수다 대치점', '익선취향', '이수가 원조', '갈치호', '광어연어', '일번지

2. 라이브러리

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
from bs4 import BeautifulSoup
import time
import pandas as pd

In [5]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# 스크롤 함수 정의
def scroll_down(driver, n):
    for _ in range(n):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

# 크롤링 시작
for region in unique_regions:
    region_name = f"서울_{region}_식당.csv"
    folder_path = f"/Users/junho/Hadoop-Classification/data/crawling/{region}"
    file_path = os.path.join(folder_path, region_name)
    
    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 이미 존재하면 스킵
    if os.path.exists(file_path):
        print(f"File {file_path} already exists. Skipping...\n")
        continue
    
    region_restaurants = [restaurant for restaurant, reg in zip(restaurant_list, data['region']) if reg == f"서울 {region}"]
    
    url_list = []
    title_list = []
    
    for restaurant in region_restaurants:
        # 크롬 드라이버 설정 및 실행
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('/Users/junho/bdp/chromedriver-mac-arm64/chromedriver')
        driver = webdriver.Chrome(options=chrome_options)
        
        try:
            # 네이버 검색 페이지 접속
            driver.get("https://search.naver.com")
            time.sleep(1)
            
            # 검색창에 지역 + 식당 이름 입력 및 검색
            search_box = driver.find_element(By.CSS_SELECTOR, 'div.search_input_box > input#query')
            search_box.send_keys(f"{region} {restaurant}")
            search_box.submit()
            time.sleep(1)
            
            driver.find_element(By.LINK_TEXT, "블로그").click()
            time.sleep(1)
            
            # 검색 결과 크롤링
            scroll_down(driver, n=1)
            articles = driver.find_elements(By.CSS_SELECTOR, "a.title_link")
            
            for article in articles:
                title = article.text
                url = article.get_attribute("href")
                
                # URL과 제목 저장
                url_list.append(url)
                title_list.append(title)
        
        except Exception as e:
            print(f"Error while processing {restaurant}: {e}")
        
        finally:
            driver.quit()
    
    # 크롤링 데이터 저장
    df = pd.DataFrame({'url': url_list, 'title': title_list})
    df.to_csv(file_path, index=False)
    print(f"Saved {file_path}")

print("Crawling completed.")


Error while processing 루프탑미드: Message: no such element: Unable to locate element: {"method":"link text","selector":"블로그"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104d0bac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000104d04314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x000000010476c4b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x00000001047b0898 cxxbridge1$string$len + 368748
4   chromedriver                        0x00000001047ea0fc cxxbridge1$string$len + 604368
5   chromedriver                        0x00000001047a50b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x00000001047a5d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x0000000104cd6e